# Goal

1. Keywords similarity as the first run filtering
2. Distance in knowledge graph as the second run filtering

In [54]:
import os
import re
import pandas as pd
import json
from src.utils import read_json
from langchain.document_loaders import PyPDFLoader
from src.FastTextRank4Word import FastTextRank4Word
from src.FastTextRank4Sentence import FastTextRank4Sentence

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

In [55]:
configs = read_json('.env/configs.json')
DATA_PATH = configs["DATA_PATH"]
DB_PATH = configs['DB_PATH']
files_list = [file for file in os.listdir(DATA_PATH) if os.path.isfile(os.path.join(DATA_PATH, file))]

def read_pdf(files_list, data_path=DATA_PATH):
    content_dict = {}

    for file_name in files_list:
        file_path = data_path + file_name
        loader = PyPDFLoader(file_path)
        content = loader.load()
        whole_pdf = ""
        for i in range(len(content)):
            whole_pdf += content[i].page_content

        whole_pdf = re.sub(r'[：。\n\s]', '', whole_pdf)
        content_dict[file_name] = whole_pdf

    return content_dict

In [56]:
pdf_contents = read_pdf(files_list)

In [57]:
cleaned_text_1 = pdf_contents[list(pdf_contents.keys())[0]]
file_name_1 = list(pdf_contents.keys())[0]
cleaned_text_1

'一糾正案文壹、被糾正機關行政院原住民族委員會及花蓮縣政府貳、案由花蓮區漁會辦理「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機採購案」，未能確實依政府採購法之規定辦理，採購作業錯誤叢生，補助機關行政院原住民族委員會及花蓮縣政府原住民行政局顯然疏於監督；又花蓮區漁會辦理該採購案，未依規定辦理驗收作業，核銷時亦未檢附初驗紀錄及驗收紀錄，惟花蓮縣政府卻仍予付款，顯未覈實辦理核銷及善盡審核監督之責；該府對於補助案件未訂定明確作業規範，以供受補助單位辦理採購案件時應行遵循事項、採購案件之監辦、經費核銷、及追蹤考核之依據；行政院原住民族委員會對於「原住民新部落運動計畫｜部落社區發展計畫｜東海岸原住民漁業輔導計畫」補助款納入花蓮縣政府預算執行，惟未要求該府訂立補助款作業規範，亦未確實監督其執行情形，而查證工作流於形式；行政院原住民族委員會遲延查復立法委員及本院函詢有關花蓮區漁會辦理衛星定位儀(三合一漁探機)ＧＰＳ採購案，行政效率不彰等情，均有重大違失，爰依監察法第二十四條規定提案糾正參、事實與理由據立法委員函轉民眾陳訴行政院原住民族委員會(下稱原民會)補助花蓮區漁會給二予漁民架設衛星導航系統疑有弊端，惟該會遲未盡查處之責，涉有違失等情經本院調查結果，行政院原住民族委員會及花蓮縣政府辦理補助花蓮區漁會採購「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機」乙案，確有下列違失一、花蓮區漁會辦理「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機採購案」，未能確實依政府採購法之規定辦理，採購作業錯誤叢生，補助機關行政院原住民族委員會及花蓮縣政府原住民行政局顯然疏於監督，核有未當，應予檢討改進(一)查行政院原住民族委員會民國(下同)九十二年度補助花蓮縣政府支付花蓮區漁會辦理部落產業發展計畫｜東海岸原住民漁業輔導計畫經費新台幣(下同)一七七萬元，並納入該府預算執行，其中花蓮區漁會辦理「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機採購案」，於九十二年十月十四日開標，由友漁企業公司以一二五萬元得標，惟經查該漁會辦理該採購案違反政府採購法諸多規定，核有下列缺失１、該漁會收受三家廠商投標文件，未於標封袋登記廠商送達方式與時間，亦無簽收紀錄可稽；審查廠商投標作業，僅於證件封袋上予以勾選，無審查之人員簽名負責；又未能提供廠商繳納押標金之紀錄憑核，對於其繳交

In [58]:
cleaned_text_2 = pdf_contents[list(pdf_contents.keys())[1]]
file_name_2 = list(pdf_contents.keys())[1]
cleaned_text_2

'一糾正案文壹、被糾正機關行政院衛生署貳、案由為行政院衛生署辦理九十至九十三年度補、捐助計畫，經該署檢討至少計一二五項計畫應以委辦而竟以補、捐助方式辦理；又其中該署補、捐助受該署監督之財團法人國家衛生研究院至少計二十三項計畫亦有應以委辦而竟以補、捐助方式辦理，核與政府採購法以及預算相關法令規定有違，且該署九十二年之前對補、捐助對象、條件、金額標準、經費用途、審查標準、作業程序、督導考核等未訂定明確、公開及客觀之規範；又該署九十至九十三年度委辦、補、捐助單位，核有連續二年度補助相同單位計有八十案（計畫）、一六三單位；連續三年度補助相同單位計有三十八案(計畫)，一○八單位；連續四年度補助相同單位計有二十四案(計畫)，一三一單位之情事，該署未依行政院所屬各機關委託研究計畫管理辦法第十三條規定，列為該署計畫成效查核重點；另九十年至九十三年該署暨所屬藥物食品檢驗局、中央健康保險局人員等至少十三人次之國外旅費計一百餘萬元，竟於接受該署所補助經費項下列支，其中六人次於返國後復未曾向行政院研考會網站登錄出國報告等，顯係規避「公教人員申請出國案件審核要點」之規定，且與公務員服務法第五條及第二十一條第四款規定意旨亦有二未合等均涉有違失，爰依法提案糾正參、事實與理由本案係立法院鄭委員三元向本院陳訴略以「行政院衛生署將應以委辦執行之經費，改由補助費方式辦理，流入『特殊利害關係』社團等單位；又該署對於補助對象無一定標準且補助金額不一，致有預算浮濫編列、運用之情形，涉有違反政府採購法及預算法等規定」本院爰輪派委員進行調查，案經九十三年十一月十八日詢問行政院衛生署副署長陳再晉、醫事處處長薛瑞元、食品處處長陳陸宏、藥政處簡任技正顏秀瓊、科技組技監許須美、企劃處簡任技正周國塘、健保小組組長洪碧蘭、會計室主任高正本、專門委員陳瑞美、科長王雪齡等相關業務主管人員，並通知審計部第三廳審計官兼廳長吳清平、稽察兼科長高金展、審計朱曼如等到院說明，爰經參酌檢送之相關資料查核竣事，茲將本案之相關缺失臚列如下一、行政院衛生署九十至九十三年度委辦、補、捐助經費較多之前二十名排名單位之經費合計數一、一九五億餘元，占該署決（預）算合計數九○•五二％，該署九十至九十三年度委辦、補、捐助經費確集中於少數單位，其中補、捐助經費計一、一八一億餘元，占該署決（預）算合計數八九•四六％，而委辦經費計一四億餘元，僅占一•○六％，部分計

In [59]:
def content_extraction(input, pattern):

    # Search using the pattern
    match = re.search(pattern, input, re.S)

    if match:
        result = match.group(1).strip()  # Use strip() to remove any leading/trailing whitespace
        return result
    else:
        print("No match found")
        pass

In [60]:
def pattern_extraction(input):

    pattern = r"壹、被糾正機關(.*)貳、案由"
    target = content_extraction(input, pattern)
    pattern = r"貳、案由(.*?)參、事實.*?理由"
    reason = content_extraction(input, pattern)
    pattern = r"參、事實.*?理由(.*)"
    fact = content_extraction(input, pattern)


    return {"target": target, "reason": reason, "fact": fact}

In [61]:
def word_sentence_extraction(input, kw_num=10, ks_num=1):

    kw_model = FastTextRank4Word(tol=0.0001, window=5)
    kw=[kw_model.summarize(input['reason'], kw_num)][0]

    ks_model = FastTextRank4Sentence(use_w2v=False,tol=0.0001)
    ks=ks_model.summarize(input['fact'], ks_num)

    return { 
        "target": f"{input['target']}",
        "reason": f"{input['reason']}",
        "fact" : f"{input['fact']}",
        "metadata" : { "kw" : f"{kw}", "ks" : f"{ks}"}
        }

In [62]:
split_data = pattern_extraction(cleaned_text_1)
result_1 = word_sentence_extraction(split_data)

In [63]:
result_1

{'target': '行政院原住民族委員會及花蓮縣政府',
 'reason': '花蓮區漁會辦理「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機採購案」，未能確實依政府採購法之規定辦理，採購作業錯誤叢生，補助機關行政院原住民族委員會及花蓮縣政府原住民行政局顯然疏於監督；又花蓮區漁會辦理該採購案，未依規定辦理驗收作業，核銷時亦未檢附初驗紀錄及驗收紀錄，惟花蓮縣政府卻仍予付款，顯未覈實辦理核銷及善盡審核監督之責；該府對於補助案件未訂定明確作業規範，以供受補助單位辦理採購案件時應行遵循事項、採購案件之監辦、經費核銷、及追蹤考核之依據；行政院原住民族委員會對於「原住民新部落運動計畫｜部落社區發展計畫｜東海岸原住民漁業輔導計畫」補助款納入花蓮縣政府預算執行，惟未要求該府訂立補助款作業規範，亦未確實監督其執行情形，而查證工作流於形式；行政院原住民族委員會遲延查復立法委員及本院函詢有關花蓮區漁會辦理衛星定位儀(三合一漁探機)ＧＰＳ採購案，行政效率不彰等情，均有重大違失，爰依監察法第二十四條規定提案糾正',
 'fact': '據立法委員函轉民眾陳訴行政院原住民族委員會(下稱原民會)補助花蓮區漁會給二予漁民架設衛星導航系統疑有弊端，惟該會遲未盡查處之責，涉有違失等情經本院調查結果，行政院原住民族委員會及花蓮縣政府辦理補助花蓮區漁會採購「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機」乙案，確有下列違失一、花蓮區漁會辦理「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機採購案」，未能確實依政府採購法之規定辦理，採購作業錯誤叢生，補助機關行政院原住民族委員會及花蓮縣政府原住民行政局顯然疏於監督，核有未當，應予檢討改進(一)查行政院原住民族委員會民國(下同)九十二年度補助花蓮縣政府支付花蓮區漁會辦理部落產業發展計畫｜東海岸原住民漁業輔導計畫經費新台幣(下同)一七七萬元，並納入該府預算執行，其中花蓮區漁會辦理「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機採購案」，於九十二年十月十四日開標，由友漁企業公司以一二五萬元得標，惟經查該漁會辦理該採購案違反政府採購法諸多規定，核有下列缺失１、該漁會收受三家廠商投標文件，未於標封袋登記廠商送達方式與時間，亦無簽收紀錄可稽；審查廠商投標作業，僅於證件封袋上予以勾選，無審查之人員簽名負責；又未能提供廠商

In [64]:
def content_organize(input):

    result = {}

    for i, (name, content) in enumerate(input.items()):
        split = pattern_extraction(content)
        temp = word_sentence_extraction(split)
        result[name] = temp

    print(f"{i+1} contents have been organized.")
    return result

In [65]:
organized_result = content_organize(dict(list(pdf_contents.items())))

76 contents have been organized.


In [66]:
df = pd.DataFrame(organized_result).transpose()
df = df.reset_index().rename(columns={'index':'document'})

In [67]:
df.shape

(76, 5)

In [68]:
df.to_csv('temp_result/organized_result_1.csv', index=False, encoding='utf-8')

In [19]:
with open('temp_result/organized_result_1.txt', 'w', encoding='utf-8') as text_file:
    json.dump(organized_result, text_file, ensure_ascii=False, indent=4)

In [159]:
vectorizer = CountVectorizer().fit_transform([result_1['metadata']['kw'], result_2['metadata']['kw']])
vectors = vectorizer.toarray()

cosine_sim = cosine_similarity(vectors)

print(f"Cosine Similarity: {cosine_sim[0][1]}")

Cosine Similarity: 0.3


In [160]:
vectors

array([[0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0]], dtype=int64)

# Knowledge graph

### Gemini

In [162]:
gemini_key = os.get()

In [163]:
def gemini_result_clean(gemini_result, gemini_key):
    url = f'https://generativelanguage.googleapis.com/v1/models/gemini-pro:generateContent?key={gemini_key}'
    headers = {'Content-Type': 'application/json'}
    data = {
        "contents": [
            {
                "parts": [{"text": """將以下內容調整成符合JSON規範的格式，以及補上entity1或是entity2，不要顯示JSON字樣""" + str(gemini_result)}]
            }
        ]
    }
    response = requests.post(url, headers=headers, json=data)
    print(f"response status_code: {response.status_code}")
    result = response.json()
    output = result['candidates'][0]['content']['parts'][0]['text']
    output = re.sub(r'[```json\n]','',output)
    
    return output

In [164]:
def gemini_generation(target, ks, gemini_key):    

    url = f'https://generativelanguage.googleapis.com/v1/models/gemini-pro:generateContent?key={gemini_key}'
    headers = {'Content-Type': 'application/json'}

    data = {
        "contents": [
            {
                "parts": [{"text": """三元組指的是：實體1--關係--實體2。
                        實體通常是特定的人、組織或物體，關係則是描述兩個實體之間的關係，例如 {"實體1":"立法委員","關係":制定,"實體2":"法律"}
                            將以下這些内容以三元組方式描述並以Json格式呈現，只列出5組。用繁體中文回答:""" + str(ks)}]
            }
        ]
    }
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # This will raise an HTTPError for bad responses (4xx and 5xx)
        
        result_dict = response.json()
        extracted_result = result_dict['candidates'][0]['content']['parts'][0]['text']
        
        if '未找到三元組' in extracted_result:
            return None
        else:
            result_str = re.sub(r'[```json\n]', '', extracted_result).replace("實體", "entity").replace("關係", "relationship").split("、")[0]
            return result_str

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - Status code: {response.status_code}")
    except Exception as err:
        print(f"Other error occurred: {err}")

    gemini_output = gemini_result_clean(result_str)
    
    return gemini_output


In [165]:
def knowledge_graph_json_generaton_gemini(input):
    output = input.copy()
    for name, value in input.items():
        target = value['target']
        ks = value['metadata']['ks']
        gemini_result = gemini_generation(target, ks, gemini_key)

        gemini_result = list(json.loads(gemini_result))[0]

        output[name]['knowledge_graph'] = gemini_result

    return output

In [166]:
def process_knowledge_graph(gemini_result):
    # Attempt to load the JSON data
    try:
        loaded_data = json.loads(gemini_result)
    except json.JSONDecodeError:
        return []

    if isinstance(loaded_data, list):
        return loaded_data
    elif isinstance(loaded_data, dict) and '三元組' in loaded_data:
        return loaded_data['三元組']
    else:
        # Unknown structure
        return []

def ensure_entity_keys(item):
    if "entity1" not in item:
        item["entity1"] = item.pop("entity", None)
    if "entity2" not in item:
        item["entity2"] = item.pop("entity", None)
    return item

def filter_and_format_knowledge_graph(gemini_result):
    knowledge_graph = process_knowledge_graph(gemini_result)

    filtered_data = [
        ensure_entity_keys(item) for item in knowledge_graph
        if all(value is not None for value in item.values())
    ]

    return filtered_data


def extract_and_organize_json(organized_result, gemini_key=gemini_key):
    output = organized_result.copy()
    for name, value in organized_result.items():
        target = value['target']
        ks = value['metadata']['ks']
        gemini_result = gemini_generation(target, ks, gemini_key)

        processed_graph = filter_and_format_knowledge_graph(gemini_result)

        output[name]['knowledge_graph'] = processed_graph

    return output

In [167]:
# with open('organized_data_1.txt', 'w', encoding='utf-8') as text_file:
#     json.dump(organized_result, text_file, ensure_ascii=False, indent=4)
# with open('organized_data_1.txt', 'r', encoding='utf-8') as text_file:
#     data = json.load(text_file)

In [172]:
organized_final_result = extract_and_organize_json(organized_result)

In [186]:
organized_final_result['34829.pdf']['knowledge_graph']

[{'entity1': '立法委員', 'relationship': '函轉', 'entity2': '民眾陳訴'},
 {'entity1': '行政院原住民族委員會', 'relationship': '補助', 'entity2': '花蓮區漁會'},
 {'entity1': '花蓮區漁會',
  'relationship': '採購',
  'entity2': '「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機」'},
 {'entity1': '行政院原住民族委員會', 'relationship': '監督', 'entity2': '花蓮區漁會'},
 {'entity1': '花蓮縣政府', 'relationship': '監督', 'entity2': '花蓮區漁會'}]

In [174]:
with open('organized_final_result.txt', 'w', encoding='utf-8') as text_file:
    json.dump(organized_final_result, text_file, ensure_ascii=False, indent=4)

In [169]:
"""
[{'entity1': '立法委員', 'relationship': '制定', 'entity2': '法律'},
 {'entity1': '聯勤總部暨所屬營工署',
  'relationship': '規劃不周未併案發包於先',
  'entity2': '林子口營區後續增設工程'},
 {'entity1': '營工署',
  'relationship': '未訂定施工所核對和地區工程處工地查驗作業時程',
  'entity2': '海軍神鷹二號新建水電工程及忠貞營區戰技館新建水電工程'},
 {'entity1': '地區工程處',
  'relationship': '完成查驗後，營工署審核流程和簽證作業時間過長',
  'entity2': '海軍神鷹二號新建水電工程及忠貞營區戰技館新建水電工程'},
 {'entity1': '營工署',
  'relationship': '承辦人員因公務差勤影響計價時效',
  'entity2': '海軍神鷹二號新建水電工程及忠貞營區戰技館新建水電工程'}]
"""

"\n[{'entity1': '立法委員', 'relationship': '制定', 'entity2': '法律'},\n {'entity1': '聯勤總部暨所屬營工署',\n  'relationship': '規劃不周未併案發包於先',\n  'entity2': '林子口營區後續增設工程'},\n {'entity1': '營工署',\n  'relationship': '未訂定施工所核對和地區工程處工地查驗作業時程',\n  'entity2': '海軍神鷹二號新建水電工程及忠貞營區戰技館新建水電工程'},\n {'entity1': '地區工程處',\n  'relationship': '完成查驗後，營工署審核流程和簽證作業時間過長',\n  'entity2': '海軍神鷹二號新建水電工程及忠貞營區戰技館新建水電工程'},\n {'entity1': '營工署',\n  'relationship': '承辦人員因公務差勤影響計價時效',\n  'entity2': '海軍神鷹二號新建水電工程及忠貞營區戰技館新建水電工程'}]\n"

In [170]:
Prompt = """
根據以下內容列出三元組，以JSON方式呈現，若文章中的提及的對象跟target是同一個的話則替會成target
target: 行政院衛生署
內容：{content}
"""

In [171]:
[
    {"subject": "政府", "predicate": "委託", "object": "行政院衛生署"},
    {"subject": "行政院衛生署", "predicate": "審慎衡酌考量", "object": "研究計畫"},
    {"subject": "行政院衛生署", "predicate": "刊登於機關網業", "object": "研究計畫主題及研究重點"},
    {"subject": "中央主管機關", "predicate": "實地查核", "object": "各機關委託研究計畫管理情形"},
    {"subject": "行政院衛生署", "predicate": "連續補助", "object": "相同單位或人員"},
    {"subject": "行政院衛生署", "predicate": "核有", "object": "連續二年度補助相同單位八十案"},
    {"subject": "行政院衛生署", "predicate": "核有", "object": "連續三年度補助相同單位三十八案"},
    {"subject": "行政院衛生署", "predicate": "核有", "object": "連續四年度補助相同單位二十四案"},
    {"subject": "行政院衛生署", "predicate": "向本院說明", "object": "補助計畫經公開徵求"},
    {"subject": "行政院衛生署", "predicate": "執行不同階段的任務", "object": "連續年度補助"},
    {"subject": "行政院衛生署", "predicate": "提出", "object": "九十至九十一年度實地查核報告"},
    {"subject": "行政院衛生署", "predicate": "提出", "object": "三項單一計畫實地查核報告"},
    {"subject": "行政院衛生署", "predicate": "補助", "object": "國立台灣師範大學林薇教授"},
    {"subject": "行政院衛生署", "predicate": "補助", "object": "生命期營養單張及指導手冊發展計畫"},
    {"subject": "行政院衛生署", "predicate": "檢送", "object": "重新編修的宣導單張及小冊"},
    {"subject": "行政院衛生署", "predicate": "查證稽核", "object": "委辦及補、捐助計畫執行情形"},
    {"subject": "行政院衛生署", "predicate": "列支", "object": "國外旅費"},
    {"subject": "行政院衛生署", "predicate": "規避", "object": "公教人員申請出國案件審核要點"}
]


[{'subject': '政府', 'predicate': '委託', 'object': '行政院衛生署'},
 {'subject': '行政院衛生署', 'predicate': '審慎衡酌考量', 'object': '研究計畫'},
 {'subject': '行政院衛生署', 'predicate': '刊登於機關網業', 'object': '研究計畫主題及研究重點'},
 {'subject': '中央主管機關', 'predicate': '實地查核', 'object': '各機關委託研究計畫管理情形'},
 {'subject': '行政院衛生署', 'predicate': '連續補助', 'object': '相同單位或人員'},
 {'subject': '行政院衛生署', 'predicate': '核有', 'object': '連續二年度補助相同單位八十案'},
 {'subject': '行政院衛生署', 'predicate': '核有', 'object': '連續三年度補助相同單位三十八案'},
 {'subject': '行政院衛生署', 'predicate': '核有', 'object': '連續四年度補助相同單位二十四案'},
 {'subject': '行政院衛生署', 'predicate': '向本院說明', 'object': '補助計畫經公開徵求'},
 {'subject': '行政院衛生署', 'predicate': '執行不同階段的任務', 'object': '連續年度補助'},
 {'subject': '行政院衛生署', 'predicate': '提出', 'object': '九十至九十一年度實地查核報告'},
 {'subject': '行政院衛生署', 'predicate': '提出', 'object': '三項單一計畫實地查核報告'},
 {'subject': '行政院衛生署', 'predicate': '補助', 'object': '國立台灣師範大學林薇教授'},
 {'subject': '行政院衛生署', 'predicate': '補助', 'object': '生命期營養單張及指導手冊發展計畫'},
 {'subject': '行政院衛生署', 'predi